In [36]:
import os
import openai
import sys
sys.path.append('../..')

from IPython.display import display, Markdown

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file 

openai.api_key  = os.environ['OPENAI_API_KEY']
# os.environ['LANGSMITH_API_KEY']
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings #para crear los embeddings https://python.langchain.com/docs/integrations/text_embedding
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter #para dividir los datos
from langchain.vectorstores import Chroma #para almacenar los vectores de embeddings
from pinecone import Pinecone #vectorstore
from langchain.document_loaders import TextLoader #para cargar texto
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain #para recuperar los documentos
from langchain.memory import ConversationBufferMemory #para crear un buffer de memoria
from langchain.chat_models import ChatOpenAI 
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
import os
import pandas as pd
import objects
import importlib #https://stackoverflow.com/questions/1254370/reimport-a-module-while-interactive
import panel as pn
import param
import datetime

In [12]:
importlib.reload(objects)


<module 'objects' from 'c:\\Users\\joaqu\\OneDrive\\Escritorio\\AI Projects\\Query_your_files\\objects.py'>

In [14]:
c=objects.cbfs()
c.build_ui().show()

Fecha:2024-01-24
Usando modelo gpt-3.5-turbo-0301
Launching server at http://localhost:49900


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'button_click', 'values': {'type': 'map', 'entries': [['model', {'id': 'p2158'}]]}}}]} 
 error: AttributeError("'NoneType' object has no attribute 'value'")
Traceback (most recent call last):
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.docu

In [34]:
def current_model():
    current_date = datetime.datetime.now().date()
    target_date = datetime.date(2024, 6, 12)
    if current_date < target_date:
        llm_name = "gpt-3.5-turbo-0301"
    else:
        llm_name = "gpt-3.5-turbo-0613"
    print(f"Fecha:{current_date}\nUsando modelo {llm_name}")
    return llm_name



def load_db(file, chain_type, k):
    
    persist_directory = f"./chroma"
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = Chroma.from_documents(docs, embeddings,persist_directory=persist_directory)
    # define retriever
    retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    llm_name=current_model()
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/tgi.pdf" 
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
    @param.depends('clr_history',)
    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=700)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('Usuario:', pn.pane.Markdown(query, width=700)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=700, style={'background-color': '#ffcc00'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    
    def clr_history(self,count=0):
        self.chat_history = []
        self.panels=[]
        return 
    



        

In [35]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning',icon='caret-right')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Escriba aqui para comenzar a chatear...',width=600)

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 
jpg_pane = pn.pane.Image( './img/convchain.jpg')
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Chatea con tus archivos🗃️')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard.show()

Fecha:2024-01-24
Usando modelo gpt-3.5-turbo-0301
Launching server at http://localhost:50828


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\langchain_core\callbacks\manager.py", line 1936, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\langchain_core\tracers\langchain.py", line 96, in __init__
    self.client = client or get_client()
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\langchain_core\tracers\langchain.py", line 56, in get_client
    _CLIENT = Client()
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\langsmith\client.py", line 301, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\joaqu\OneDrive\Escritorio\Projects\Building_Systems_ChatGPT\llm_env\lib\site-packages\langsmith\client.p